In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))


In [10]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

llm = ChatOpenAI(model=OPENAI_LLM_MODEL, temperature=0.2, openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)
vector_store = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings,
    pinecone_api_key=PINECONE_API_KEY
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

def recommand_dishs(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """

            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.

        """)
    ])

    template = [{"text": query["text"]}]
    if query["image_urls"]:
        template += [{"image_url": url} for url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"text": query["text"], "image_urls": query["image_urls"]})

In [7]:
response = recommand_dishs({
    "text": "이 와인에 어울리는 요리를 추천해줘",
    "image_urls": [
        "https://images.vivino.com/thumbs/pGBPuKapTwGBpeJmdq7uTw_pb_x600.png"
    ]
})

print(response)

Chateau Musar는 레바논의 유명한 와인으로, 복합적인 맛과 향을 지니고 있습니다. 이 와인은 일반적으로 진한 과일 맛, 스파이스, 그리고 약간의 흙내음이 특징입니다. 

이 와인에 어울리는 요리로는 다음을 추천합니다:

1. **양고기 스튜**: 양고기의 풍미가 Chateau Musar의 복합적인 맛과 잘 어우러집니다.
2. **그릴에 구운 야채와 퀴노아 샐러드**: 구운 야채의 스모키한 맛이 와인의 깊은 맛을 강조합니다.
3. **버섯 리조또**: 버섯의 풍미가 와인의 earthy notes와 잘 맞아떨어집니다.
4. **스파이스가 들어간 인도 커리**: 와인의 스파이스와 커리의 향신료가 조화를 이룹니다.

이 요리들은 Chateau Musar의 풍부한 맛을 더욱 돋보이게 해줄 것입니다.


In [8]:
def describe_dish_flavor(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.

        """)
    ])

    template = [{"text": query["text"]}]
    if query["image_urls"]:
        template += [{"image_url": url} for url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"text": query["text"], "image_urls": query["image_urls"]})

In [9]:
response = describe_dish_flavor({
    "text": "이 요리의 이름과 맛과 향과 같은 특징을 한 문장으로 설명해줘.",
    "image_urls": [
        "https://media01.stockfood.com/largepreviews/MzkyOTA0Mjky/12674332-Sweet-soy-salmon-with-rice-and-bok-choy-China.jpg"
    ]
})

print(response)

이 요리는 달콤하고 짭짤한 맛의 글레이즈가 입혀진 연어와 부드러운 흰 쌀밥, 아삭한 채소가 조화를 이루며, 향긋한 파의 향이 더해져 풍부하고 균형 잡힌 맛을 제공합니다.


In [11]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor,
        k=2
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n\n".join([doc.page_content for doc in results])
    }

In [13]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(search_wine)
response = runnable.invoke("달콤한 맛을 가진 와인")

print(response["dish_flavor"])
print(response["wine_reviews"])

달콤한 맛을 가진 와인
: 11089
country: US
description: Ripe hay, vanilla, and apricot mark the nose; deep orange flavors with cinnamon spice notes and a very smooth mouthfeel are the already promising attributes of this infant. All the right elements are there, and in good balance. Best cellared for three to five years.
designation: 
points: 93
price: 75.0
province: California
region_1: Napa Valley
region_2: Napa
taster_name: 
taster_twitter_handle: 
title: Dolce 1998 White (Napa Valley)
variety: White Blend
winery: Dolce

: 6319
country: US
description: Aromas of candy corn, spice and almond butter are followed by full-bodied, tropical-fruit flavors that show a pleasing sense of richness and intensity. It's a delicious example of the variety, with a lingering finish.
designation: Barrel Fermented
points: 91
price: 25.0
province: Washington
region_1: Columbia Valley (WA)
region_2: Columbia Valley
taster_name: Sean P. Sullivan
taster_twitter_handle: @wawinereport
title: Ashan 2016 Barrel Ferment

In [14]:
runnable1 = RunnableLambda(describe_dish_flavor)
runnable2 = RunnableLambda(search_wine)

chain = runnable1 | runnable2

response = chain.invoke({
    "text": "이 요리의 이름과 맛과 향과 같은 특징을 한 문장으로 설명해줘.",
    "image_urls": [
        "https://media01.stockfood.com/largepreviews/MzkyOTA0Mjky/12674332-Sweet-soy-salmon-with-rice-and-bok-choy-China.jpg"
    ]
})  

print(response["dish_flavor"], "\n")
print(response["wine_reviews"])

이 요리는 달콤하고 짭짤한 맛이 조화롭게 어우러진 글레이즈드 연어와 부드러운 밥, 아삭한 채소가 함께 제공되어 풍부한 식감과 신선한 향을 느낄 수 있는 한 그릇입니다. 

: 34010
country: France
description: A bouquet of summer desserts—strawberry soup, apricot pie and lemon meringue pie—pull you into the glass. Although not quite as nuanced, the palate delivers enticing apricot and wet stone notes. The unctuous palate and noticeable residual sugar will partner well with fish in an Asian-fusion preparation.
designation: Herrenweg
points: 91
price: 22.0
province: Alsace
region_1: Alsace
region_2: 
taster_name: 
taster_twitter_handle: 
title: Domaine Ehrhart 2006 Herrenweg Riesling (Alsace)
variety: Riesling
winery: Domaine Ehrhart

: 39707
country: Germany
description: Savory spice and smoke tones accent fresh lemon, grapefruit and quince notes on this delicate off-dry Riesling. Easy drinking and widely appealing, it's nicely concentrated on the palate with a brisk, blossomy finish.
designation: 
points: 87
price: 14.0
province: Mosel
region_1: 
region_2: 
tas

In [15]:
def recommand_wine(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:
            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.
            Role:
            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.
            Examples:
            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.
            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.
            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.
            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.
            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
        """),
        ("human", """
            와인 페어링 추천에 아래의 요리의 풍미와 와인 리뷰를 참고해 한글로 답변해 주세요요.
            요리의 풍미:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """)
    ])

    chain = prompt | llm | StrOutputParser()

    return chain.invoke(query)

In [16]:
runnable1 = RunnableLambda(describe_dish_flavor)
runnable2 = RunnableLambda(search_wine)
runnable3 = RunnableLambda(recommand_wine)

chain = runnable1 | runnable2 | runnable3

response = chain.invoke({
    "text": "이 요리의 이름과 맛과 향과 같은 특징을 한 문장으로 설명해줘.",
    "image_urls": [
        "https://media01.stockfood.com/largepreviews/MzkyOTA0Mjky/12674332-Sweet-soy-salmon-with-rice-and-bok-choy-China.jpg"
    ]
})

print(response)

이 요리는 달콤하고 짭짤한 맛의 글레이즈가 입혀진 연어와 부드러운 밥, 아삭한 청경채, 그리고 향긋한 파의 풍미가 조화를 이루고 있습니다. 이러한 풍미를 고려할 때, 두 가지 리슬링 와인을 추천드립니다.

1. **Domaine Ehrhart 2006 Herrenweg Riesling (Alsace)**  
   - **국가**: 프랑스  
   - **설명**: 여름 디저트의 향이 가득한 이 와인은 딸기 수프, 살구 파이, 레몬 머랭 파이의 향이 느껴집니다. 입 안에서는 유혹적인 살구와 젖은 돌의 노트가 느껴지며, 부드러운 질감과 눈에 띄는 잔여 당도가 아시아 퓨전 요리와 잘 어울립니다.  
   - **포인트**: 91  
   - **가격**: 22.0  

   이 리슬링은 연어의 달콤한 글레이즈와 잘 어우러져 풍미를 더욱 강조해 줄 것입니다.

2. **Coquelicot 2007 Estate Riesling (Santa Ynez Valley)**  
   - **국가**: 미국  
   - **설명**: 독일 카비넷 스타일로 만들어진 이 리슬링은 산도가 깔끔하고 매끄럽습니다. 복숭아, 파인애플, 정원화, 그리고 석유의 미네랄 노트가 조화를 이루며, 팬에 튀긴 송어와 잘 어울립니다.  
   - **포인트**: 88  
   - **가격**: 22.0  

   이 와인은 연어와 함께 즐기기에 적합하며, 아삭한 청경채와의 조화도 기대할 수 있습니다.

이 두 가지 리슬링은 요리의 풍미와 잘 어울리며, 각각의 특성이 요리를 더욱 돋보이게 할 것입니다. 즐거운 식사 되세요!
